In [107]:
# You need to replace these placeholders with your keys

consumer_key = 'CONSUMER_KEY_PLACEHOLDER'
consumer_secret = 'CONSUMER_SECRET_PLACEHOLDER'
access_token = 'ACCESS_TOKEN_PLACEHOLDER'
access_secret = 'ACCESS_SECRET_PLACEHOLDER'

In [108]:
# importing libraries
import pandas as pd
import requests
import tweepy
import numpy as np
import time
import json
import os
import contextlib

In [109]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [110]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [111]:
# retrieve image predictions
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

# request from the url
req = requests.get(url)
if req.status_code == requests.codes.ok:
    req.encoding = 'utf-u'
    with open('image_predictions.txt', 'w') as infile:
        # write the text into the txt file
        infile.write(req.text)
        
# read the content using pandas 
df_image_predictions = pd.read_csv("image_predictions.txt", sep= '\t')

df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [112]:
df_image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [113]:
# get the tweet IDs from df_archive and df_image_predictions
tweet_IDs = np.array(df_archive['tweet_id'].values, dtype = str)
tweet_IDs = np.concatenate((np.array(df_image_predictions['tweet_id'].values, dtype = str), tweet_IDs))
tweet_IDs = np.unique(tweet_IDs)
tweet_IDs

array(['666020888022790149', '666029285002620928', '666033412701032449',
       ..., '891815181378084864', '892177421306343426',
       '892420643555336193'],
      dtype='<U21')

In [114]:
#create the api instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(),
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

# create the lists where we will store the results from the twitter api
tweets = []
errors = []

for tweet_id in tweet_IDs:
    start = time.time()
    try:
        result = api.get_status(tweet_id, tweet_mode = 'extended')
        # Uncomment the lines below to see a JSON dump of one result
        # print(json.dumps(result, indent = 2))
        # break
        
        favorites = result['favorite_count']
        retweets = result['retweet_count']
        
        tweets.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets)})
    
    except Exception as e:
        errors.append(tweet_id)
    end = time.time()
    request_time = end - start
    print("time required: ", request_time)

time required:  0.2721731662750244
time required:  0.29801416397094727
time required:  0.25975990295410156
time required:  0.27326226234436035
time required:  0.2758307456970215
time required:  0.2509009838104248
time required:  0.2517261505126953
time required:  0.36673688888549805
time required:  0.286118745803833
time required:  0.2756810188293457
time required:  0.28613996505737305
time required:  0.5287537574768066
time required:  0.29094409942626953
time required:  0.3021278381347656
time required:  0.28908491134643555
time required:  0.29513120651245117
time required:  0.28769922256469727
time required:  0.2520451545715332
time required:  0.2902369499206543
time required:  0.2730109691619873
time required:  0.27274584770202637
time required:  0.27469491958618164
time required:  0.2908210754394531
time required:  0.2779572010040283
time required:  0.2557799816131592
time required:  0.2808690071105957
time required:  0.2683291435241699
time required:  0.2519512176513672
time requi

time required:  0.27533912658691406
time required:  0.3206319808959961
time required:  0.2591390609741211
time required:  0.30227184295654297
time required:  0.2625081539154053
time required:  0.3410201072692871
time required:  0.3943769931793213
time required:  0.2697012424468994
time required:  0.2780640125274658
time required:  0.29247570037841797
time required:  0.25500011444091797
time required:  0.2743680477142334
time required:  0.2894911766052246
time required:  0.2650160789489746
time required:  0.26399683952331543
time required:  0.3006420135498047
time required:  0.27074575424194336
time required:  0.26330089569091797
time required:  0.28612589836120605
time required:  0.25324201583862305
time required:  0.2629969120025635
time required:  0.3234219551086426
time required:  0.34358716011047363
time required:  0.3227579593658447
time required:  0.2998778820037842
time required:  0.2560303211212158
time required:  0.3216259479522705
time required:  0.277390718460083
time requir

time required:  0.2838931083679199
time required:  0.328732967376709
time required:  0.26806187629699707
time required:  0.26181912422180176
time required:  0.2638702392578125
time required:  0.6736550331115723
time required:  0.2970271110534668
time required:  0.29026293754577637
time required:  0.3097388744354248
time required:  0.2742919921875
time required:  0.2826559543609619
time required:  0.27530598640441895
time required:  0.3495056629180908
time required:  0.3773362636566162
time required:  0.3571200370788574
time required:  0.2869398593902588
time required:  0.27427196502685547
time required:  0.36272692680358887
time required:  0.2854461669921875
time required:  0.30964183807373047
time required:  0.34780001640319824
time required:  0.26422810554504395
time required:  0.26133108139038086
time required:  0.30418896675109863
time required:  0.269420862197876
time required:  0.3456418514251709
time required:  0.27316904067993164
time required:  0.32399797439575195
time require

time required:  0.3164961338043213
time required:  0.27285099029541016
time required:  0.3054389953613281
time required:  0.2707350254058838
time required:  0.26633596420288086
time required:  0.26343607902526855
time required:  0.3910820484161377
time required:  0.2891528606414795
time required:  0.25559210777282715
time required:  0.2542860507965088
time required:  0.2680470943450928
time required:  0.304882287979126
time required:  0.27893924713134766
time required:  0.2772080898284912
time required:  0.3341710567474365
time required:  0.27448391914367676
time required:  0.27162718772888184
time required:  0.28490424156188965
time required:  0.3271012306213379
time required:  0.3060600757598877
time required:  0.32689380645751953
time required:  0.2955141067504883
time required:  0.31606316566467285
time required:  0.3245677947998047
time required:  0.25533080101013184
time required:  0.2638211250305176
time required:  0.2547612190246582
time required:  0.2980160713195801
time requi

time required:  0.2641329765319824
time required:  0.40512800216674805
time required:  0.30840086936950684
time required:  0.2803351879119873
time required:  0.3059518337249756
time required:  0.2704489231109619
time required:  0.3019440174102783
time required:  0.2563588619232178
time required:  0.27686405181884766
time required:  0.27086782455444336
time required:  0.26634883880615234
time required:  0.30935215950012207
time required:  0.2704899311065674
time required:  0.3645749092102051
time required:  0.30875706672668457
time required:  0.25544285774230957
time required:  0.29983997344970703
time required:  0.2600560188293457
time required:  0.26708030700683594
time required:  0.28191494941711426
time required:  0.2576181888580322
time required:  0.25917816162109375
time required:  0.33608293533325195
time required:  0.3204841613769531
time required:  0.27707695960998535
time required:  0.2621760368347168
time required:  0.2815718650817871
time required:  0.2597959041595459
time r

...

time required:  0.262423038482666
time required:  0.3029639720916748
time required:  0.32177186012268066
time required:  0.280900239944458
time required:  0.3056061267852783
time required:  0.27566003799438477
time required:  0.31232285499572754
time required:  0.3001902103424072
time required:  0.2642190456390381
time required:  0.2707059383392334
time required:  0.2841191291809082
time required:  0.3040440082550049
time required:  0.26693105697631836
time required:  0.2797839641571045
time required:  0.2601590156555176
time required:  0.28499889373779297
time required:  0.307661771774292
time required:  0.2748599052429199
time required:  0.349215030670166
time required:  0.26930904388427734
time required:  0.26441502571105957
time required:  0.3454310894012451
time required:  0.2718210220336914
time required:  0.31357312202453613
time required:  0.37175583839416504
time required:  0.2664179801940918
time required:  0.2980020046234131
time required:  0.26937294006347656
time required:

time required:  0.27112913131713867
time required:  0.27367091178894043
time required:  0.2623481750488281
time required:  0.44741392135620117
time required:  0.2786130905151367
time required:  0.2809607982635498
time required:  0.30568695068359375
time required:  0.31511807441711426
time required:  0.26392388343811035
time required:  0.33057188987731934
time required:  0.31836891174316406
time required:  0.28107523918151855
time required:  0.36098289489746094
time required:  0.2748379707336426
time required:  0.27367591857910156
time required:  0.2629261016845703
time required:  0.27855896949768066
time required:  0.32854771614074707
time required:  0.28501200675964355
time required:  0.2721710205078125
time required:  0.2651529312133789
time required:  0.3441770076751709
time required:  0.3534708023071289
time required:  0.3144819736480713
time required:  0.29192423820495605
time required:  0.33521485328674316
time required:  0.2600231170654297
time required:  0.2705521583557129
time

time required:  0.3383018970489502
time required:  0.6458301544189453
time required:  0.2621729373931885
time required:  0.2754828929901123
time required:  0.28519511222839355
time required:  0.37304186820983887
time required:  0.28086090087890625
time required:  0.2817549705505371
time required:  0.2589590549468994
time required:  0.2622199058532715
time required:  0.2780756950378418
time required:  0.3871340751647949
time required:  0.2789602279663086
time required:  0.2808060646057129
time required:  0.3167111873626709
time required:  0.2925889492034912
time required:  0.29048871994018555
time required:  0.29537415504455566
time required:  0.2960820198059082
time required:  0.2789781093597412
time required:  0.2716548442840576
time required:  0.2657301425933838
time required:  0.27178287506103516
time required:  0.27569103240966797
time required:  0.2653670310974121
time required:  0.2805182933807373
time required:  0.30469608306884766
time required:  0.28155088424682617
time requir

time required:  0.25742387771606445
time required:  0.2726001739501953
time required:  0.2837698459625244
time required:  0.2989339828491211
time required:  0.26262617111206055
time required:  0.2644991874694824
time required:  0.29903388023376465
time required:  0.2683122158050537
time required:  0.28874993324279785
time required:  0.37786388397216797
time required:  0.2606191635131836
time required:  0.28819704055786133
time required:  0.26019906997680664
time required:  0.2540931701660156
time required:  0.3134768009185791
time required:  0.25987672805786133
time required:  0.28661298751831055
time required:  0.29537010192871094
time required:  0.3111696243286133
time required:  0.29662108421325684
time required:  0.2691657543182373
time required:  0.2886080741882324
time required:  0.2714810371398926
time required:  0.2917029857635498
time required:  0.2719409465789795
time required:  0.3015780448913574
time required:  0.30549120903015137
time required:  0.2898378372192383
time req

time required:  0.26304197311401367
time required:  0.2602829933166504
time required:  0.3154008388519287
time required:  0.2830040454864502
time required:  0.26708412170410156
time required:  0.2953817844390869
time required:  0.3185420036315918
time required:  0.6172850131988525
time required:  0.26214599609375
time required:  0.35714101791381836
time required:  0.2651972770690918
time required:  0.32941102981567383
time required:  0.26928019523620605
time required:  0.26717424392700195
time required:  0.40108799934387207
time required:  0.2674899101257324
time required:  0.3102989196777344
time required:  0.29788708686828613
time required:  0.33736300468444824
time required:  0.2651042938232422
time required:  0.23368406295776367
time required:  0.35689783096313477
time required:  0.3005948066711426
time required:  0.2613809108734131
time required:  0.2643089294433594
time required:  0.2979390621185303
time required:  0.2823460102081299
time required:  0.2826721668243408
time requir

In [115]:
print("The number of results from twitter", len(tweets))
print("The number of errors", len(errors))

The number of results from twitter 2348
The number of errors 8


In [118]:
with contextlib.suppress(FileNotFoundError):
    os.remove('tweet_json.txt')

df_json_tweets = pd.DataFrame(tweets, columns = ['tweet_id', 'favorites', 'retweets'])

df_json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)
df_json_tweets.head()

,tweet_id,favorites,retweets
0,666020888022790149,2528,523
1,666029285002620928,131,47
2,666033412701032449,126,46
3,666044226329800704,305,143
4,666049248165822465,110,40
